### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# Desarrollo desafío 1




### Se cargan librerías

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from utils import analyze_document_similarity, find_top_similar_terms

import numpy as np


### Carga de los datos

In [26]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

### 1. Vectorización de documentos
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

Se inicializa el vectorizador TF-IDF y se entrena

In [27]:
tfidfvect = TfidfVectorizer()

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [28]:
print(type(X_train))
print(f'Shape: {X_train.shape}')
print(f'Number of documents: {X_train.shape[0]}')
print(f'Vocabulary size: {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
Shape: (11314, 101631)
Number of documents: 11314
Vocabulary size: 101631


Se obtienen los identificadores de cinco documentos al aza

In [46]:
np.random.seed(30)

documents_idx = np.random.choice(X_train.shape[0], 5)
print(f"Randomly selected documents indices: {documents_idx}")

Randomly selected documents indices: [5925 4517  500  919 4859]


### Análisis del documento 5925

In [50]:
main_idx = 5925

# Calculate the cosine similarity between the main document and all the other documents
cos_sim = cosine_similarity(X_train[main_idx], X_train)[0]

# Get the indices of the most similar documents
most_similar_idx = np.argsort(cos_sim)[::-1][1:6]

print(f'Most similar documents to document {main_idx}: {most_similar_idx}')

Most similar documents to document 5925: [8379 1205 1589 7206 1706]


Se muestra el contenido del documento que hemos seleccionado

In [51]:
print(f'Class for document {main_idx}: {newsgroups_train.target_names[y_train[main_idx]]}')
print(f'Content:\n{newsgroups_train.data[main_idx]}')

Class for document 5925: rec.sport.baseball
Content:

Why not?  Brooks Robinson is a defensive liability too, and Ted
Williams is a weak hitter.  Even great players decline as they age.


El documento seleccionado parece estar relacionado con comentarios deportivos acerca de jugadores de basket.

Se analizan los documentos más similares

In [53]:
analyze_document_similarity(most_similar_idx,
                            newsgroups_train.data,
                            newsgroups_train.target,
                            newsgroups_train.target_names,
                            cos_sim, main_idx)
 


Top 5 Similar Documents:
--------------------------------------------------

Analyzing Document 8379
Content:

That's rich... Ozzie Smith a defensive liability...

Document Class: rec.sport.baseball
Reference Document Class: rec.sport.baseball

Analyzing Document 1205
Content:


Gretzky, Lemieux, Gilmour etc do not play the role of checking centreman.
They play an offensive role as opposed to a defensive one.  If they
were used as defensive centres it would be a waste of th

Document Class: rec.sport.hockey
Reference Document Class: rec.sport.baseball

Analyzing Document 1589
Content:
Don't knock Vaughn for being a spring training .400 hitter
but a .250 regular season hitter.
Around 30 games played isn't an indication of how good any
hitter is, and the quality of pitching is way do

Document Class: rec.sport.baseball
Reference Document Class: rec.sport.baseball

Analyzing Document 7206
Content:

Why?  Do you expect him to remain the best shortstop in the game until
he reaches his seve

Relación entre los textos:

Documentos de béisbol: La mayoría de los documentos analizados están clasificados bajo la categoría rec.sport.baseball, que corresponde a discusiones sobre béisbol. Algunos mencionan jugadores y temas específicos del deporte, como Ozzie Smith o Vaughn.

Documento de hockey: Uno de los documentos está clasificado como rec.sport.hockey, y su contenido trata sobre jugadores de hockey, como Gretzky y Lemieux, en roles ofensivos y defensivos.

Similitud temática: Aunque uno de los documentos pertenece a la categoría de hockey, los demás están enfocados en béisbol. Todos comparten el contexto deportivo, pero con énfasis en diferentes deportes.

La similitud entre estos documentos radica en el contexto deportivo general, aunque algunos se centran en diferentes deportes. La mayoría están alineados en la temática del béisbol.

### Análisis del documento 4517

Se obtienen los índices de los documentos mas similares al doumento 4517

In [54]:
main_idx = 4517

# Calculate the cosine similarity between the main document and all the other documents
cos_sim = cosine_similarity(X_train[main_idx], X_train)[0]

# Get the indices of the most similar documents
most_similar_idx = np.argsort(cos_sim)[::-1][1:6]

print(f'Most similar documents to document {main_idx}: {most_similar_idx}')

Most similar documents to document 4517: [ 2757 10703  4710  5509  4637]


Se muestra el contenido del documento seleccionado

In [55]:
print(f'Class for document {main_idx}: {newsgroups_train.target_names[y_train[main_idx]]}')
print(f'Content:\n{newsgroups_train.data[main_idx]}')

Class for document 4517: comp.os.ms-windows.misc
Content:







1) make sure your hard drive is defragmented. This will speed up more than 
   just windows BTW.  Use something like Norton's or PC Tools.
2) I _think_ that leaving the wall paper out will use less RAM and therefore
   will speed up your machine but I could very will be wrong on this.
There's a good chance you've already done this but if not it may speed things
up.  good luck
				Morgan Bullard mb4008@coewl.cen.uiuc.edu
					  or   mjbb@uxa.cso.uiuc.edu


El contenido del documento aborda temas relacionados con la optimización del rendimiento en computadoras que usan Windows. Los puntos clave que se tocan incluyen:

- Desfragmentación del disco duro
- Uso de recursos del sistema
- Consejos generales de optimización

Se analizan documentos similares

In [56]:
analyze_document_similarity(most_similar_idx,
                            newsgroups_train.data,
                            newsgroups_train.target,
                            newsgroups_train.target_names,
                            cos_sim, main_idx)


Top 5 Similar Documents:
--------------------------------------------------

Analyzing Document 2757
Content:

I was wondering if any one knew how the various hard drive compression utilities work.  My hard
drive is getting full and I don't want to have to buy a new one.  What I'm intrested in is speed
,ease 

Document Class: comp.os.ms-windows.misc
Reference Document Class: comp.os.ms-windows.misc

Analyzing Document 10703
Content:



That's a standby unit not a UPS, otherwise there would be no interuption.
Many standby units are labled as UPS's though.  
 			Morgan Bullard mb4008@coewl.cen.uiuc.edu
				   or  mjbb@uxa.cso.uiuc.

Document Class: sci.electronics
Reference Document Class: comp.os.ms-windows.misc

Analyzing Document 4710
Content:


Your machine will run at whatever the bus is jumpered to/CMOS is set to
(usually wait states) regardless of what speed RAM is installed.  No
motherboard can sense the speed of the RAM installed, unl

Document Class: comp.sys.ibm.pc.hardware
R

El análisis de similitud entre los documentos revela que, aunque todos los textos pertenecen a distintas categorías temáticas, comparten elementos relacionados con tecnología y hardware, pero desde diferentes enfoques. Se observan las siguientes similitudes y diferencias:

Similitudes:
Temática tecnológica: Todos los documentos tocan temas relacionados con la tecnología, el hardware o el funcionamiento técnico de sistemas. Esto incluye preguntas sobre hardware de computadoras, compresión de discos o configuraciones de hardware.
Consejos técnicos: Varios textos ofrecen o solicitan soluciones o explicaciones técnicas, como en el caso de las utilidades de compresión de disco, la velocidad de la RAM, o la discusión sobre unidades de energía UPS.

Referencias a configuraciones o componentes: Los documentos se refieren a configuraciones de hardware, como la velocidad del bus o la comparación entre arquitecturas de procesadores, lo cual es consistente con la categoría "comp.os.ms-windows.misc".

Diferencias:
Categoría de los documentos: Aunque el documento de referencia pertenece a "comp.os.ms-windows.misc", varios documentos analizados pertenecen a diferentes categorías, como sci.electronics, comp.sys.ibm.pc.hardware, comp.sys.mac.hardware y rec.motorcycles. Esto sugiere que aunque hay similitud temática, los tópicos específicos varían desde sistemas operativos de Windows hasta discusiones sobre motocicletas.

Enfoque del contenido: Algunos documentos están más centrados en el hardware (por ejemplo, velocidad de RAM o benchmarks de procesadores), mientras que otros tocan temas más amplios, como la energía ininterrumpida o incluso la mecánica de motocicletas.
Relación general:

Todos los documentos están conectados a la tecnología o la optimización del rendimiento de sistemas, pero cubren un rango amplio de subtemas, desde software hasta hardware, y en un caso incluso fuera del ámbito computacional (motocicletas). Aunque hay una base común en tecnología, los detalles y las categorías varían bastante.

### Análisis del documento 500  

Se obtienen los índices de los documentos mas similares al doumento 500
 

In [57]:
main_idx = 500

# Calculate the cosine similarity between the main document and all the other documents
cos_sim = cosine_similarity(X_train[main_idx], X_train)[0]

# Get the indices of the most similar documents
most_similar_idx = np.argsort(cos_sim)[::-1][1:6]

print(f'Most similar documents to document {main_idx}: {most_similar_idx}')

Most similar documents to document 500: [3174 5786 9867  576 6676]


Se muestra el contenido del documento seleccionado

In [58]:
print(f'Class for document {main_idx}: {newsgroups_train.target_names[y_train[main_idx]]}')
print(f'Content:\n{newsgroups_train.data[main_idx]}')

Class for document 500: comp.os.ms-windows.misc
Content:

I got one from Microsoft tech support.



El documento está clasificado en la categoría comp.os.ms-windows.misc, lo que indica que está relacionado con discusiones generales sobre Windows o problemas técnicos asociados con el sistema operativo.

Se analizan documentos similares

In [59]:
analyze_document_similarity(most_similar_idx,
                            newsgroups_train.data,
                            newsgroups_train.target,
                            newsgroups_train.target_names,
                            cos_sim, main_idx)


Top 5 Similar Documents:
--------------------------------------------------

Analyzing Document 3174
Content:
This probably is in a FAQ somewhere, but....

I'm looking for Microsoft's internal speaker sound driver for Windows.

Should be at Microsoft's FTP site, but I can't remember the name of the site...

Document Class: comp.os.ms-windows.misc
Reference Document Class: comp.os.ms-windows.misc

Analyzing Document 5786
Content:
FOR SALE !!!

      DECpc 325sxLP

   It's in very good condition, used for one year. It has

      - 25 Mhz Intel 386
      - 52 MB Hard Disk
      - Super Color VGA Monitor
      - 2-button mouse
  

Document Class: comp.sys.ibm.pc.hardware
Reference Document Class: comp.os.ms-windows.misc

Analyzing Document 9867
Content:
As quoted from <1993Apr17.025258.7013@microsoft.com> by anthonyf@microsoft.com (Anthony Francisco):


That's life.  First you marry Imelda Marcos, then you die! :)


And that's the HARD stuff to copy!

Document Class: talk.politics.guns
Re

Análisis de Similitudes:

Los textos comparten un enfoque común en temas relacionados con la tecnología y el soporte técnico. Hay un énfasis en la búsqueda de soluciones a problemas de hardware y software, así como en la necesidad de asistencia técnica. Algunos textos discuten la adquisición o venta de hardware, mientras que otros abordan problemas específicos y la comunicación con el soporte técnico.

Conclusión:

La discusión refleja un interés general en la resolución de problemas tecnológicos, con un enfoque en la interacción con el soporte técnico y el manejo de equipos informáticos. Esta variedad de temas resalta la importancia de la comunidad en la búsqueda de soluciones y el intercambio de información técnica.

### Analisis del documento 919

Se obtienen los índices mas similares al documento 919

In [60]:
main_idx = 919

# Calculate the cosine similarity between the main document and all the other documents
cos_sim = cosine_similarity(X_train[main_idx], X_train)[0]

# Get the indices of the most similar documents
most_similar_idx = np.argsort(cos_sim)[::-1][1:6]

print(f'Most similar documents to document {main_idx}: {most_similar_idx}')

Most similar documents to document 919: [9623 6437 1292 3282 7286]


Se muestra el contenido del documento seleccionado

In [61]:
print(f'Class for document {main_idx}: {newsgroups_train.target_names[y_train[main_idx]]}')
print(f'Content:\n{newsgroups_train.data[main_idx]}')

Class for document 919: talk.politics.mideast
Content:
Accounts of Anti-Armenian Human Right Violatins in Azerbaijan #009
                 Prelude to Current Events in Nagorno-Karabakh

      +-----------------------------------------------------------------+
      |                                                                 |
      | There were about six burned people in there, and the small      |
      | corpse of a burned child. It was gruesome. I suffered a         |
      | tremendous shock. There were about ten people there, but the    |
      | doctor on duty said that because of the numbers they were being |
      | taken to Baku. There was a woman's corpse there too, she had    |
      | been . . . well, there was part of a body there . . . a         |
      | hacked-off part of a woman's body. It was something terrible.   |
      |                                                                 |
      +-----------------------------------------------------------------+


Este documento ilustra una tragedia humanitaria, destacando los efectos devastadores del conflicto étnico y la violencia que puede surgir en tiempos de tensión. Además, enfatiza la necesidad de memoria y justicia para las víctimas de tales atrocidades, que siguen marcando la historia y la identidad de los pueblos involucrados. Categorizado en temas políticos de medio oriente

Se analizan documentos similares

In [62]:
analyze_document_similarity(most_similar_idx,
                            newsgroups_train.data,
                            newsgroups_train.target,
                            newsgroups_train.target_names,
                            cos_sim, main_idx)


Top 5 Similar Documents:
--------------------------------------------------

Analyzing Document 9623
Content:
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+


Document Class: talk.politics.mideast
Reference Document Class: talk.politics.mideast

Analyzing Document 6437
Content:
Accounts of Anti-Armenian Human Rights Violations in Azerbaijan #007
                 Prelude to Current Events in Nagorno-Karabakh


 +----------------------------------------------------------------

Document Class: talk.politics.mideast
Reference Document Class: talk.politics.mideast

Analyzing Document 1292
Content:
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #008 Part B
                 Prelude to Current Events in Nagorno-Karabakh

				(Part B of #008)

      +--------------------------------

Document Class: talk.politics.mideast
Refer

Analisis de Similitudes:

Los documentos analizados se centran en las violaciones de derechos humanos contra los armenios en Azerbaiyán, especialmente en el contexto de los recientes acontecimientos en la región de Nagorno-Karabaj. La mayoría de los textos siguen una estructura similar, con encabezados que mencionan explícitamente las violaciones y su relación con la situación geopolítica actual. Esta consistencia en el enfoque refleja la gravedad del tema y la necesidad de documentar los abusos sistemáticos que han tenido lugar.

Además, algunos de los documentos forman parte de una serie numerada, lo que sugiere un análisis más extenso y continuo de la problemática. La clasificación de todos los textos bajo la categoría talk.politics.mideast resalta su conexión con discusiones políticas sobre el Medio Oriente, subrayando la relevancia de las violaciones de derechos humanos en el marco del conflicto entre armenios y azeríes.

### Analisis del documento 4859

Se obtienen los índices mas similares al documento 4859

In [63]:
main_idx = 4859

# Calculate the cosine similarity between the main document and all the other documents
cos_sim = cosine_similarity(X_train[main_idx], X_train)[0]

# Get the indices of the most similar documents
most_similar_idx = np.argsort(cos_sim)[::-1][1:6]

print(f'Most similar documents to document {main_idx}: {most_similar_idx}')

Most similar documents to document 4859: [5745 9681 1128 3864 1459]


Se muestra el contenido del documento seleccionado

In [64]:
print(f'Class for document {main_idx}: {newsgroups_train.target_names[y_train[main_idx]]}')
print(f'Content:\n{newsgroups_train.data[main_idx]}')

Class for document 4859: comp.graphics
Content:
================ ------------------------------------------------------------
 |||||||| SciP+Fi  ction set in C-Sci\programming environs list by Ian Feldman
 ..........:::::: ---------------------------------------- ---- --------------
 Written by:_____ _Book Title_; publisher'year, pp         v2.7           ISBN
 -----------       =============================== ------- ==== ##############
     John Brunner _Shockwave Rider_; Ray/Ballantine'84 $5_______ 0-345-32431-5
                   "cracking the net to free information for the common good"
      Pat Cadigan _Mindplayers_; ("an absolute must-have" --Bruce Sterling)
      Pat Cadigan _Synners_; Bantam $5; (virtual reality)_______ 0-553-28254-9
 Orson Scott Card _Lost Boys_; Harper Collins'92; (programmer and family     \
                   encounters strange events in North Carolina)
     Denise Danks _Frame Grabber_; St.Martin's, hrdb [GBP]17____ 0-312-08786-1
                   compu

El documento  es una lista de libros de ciencia ficción que exploran temas tecnológicos como inteligencia artificial, realidad virtual y ciberespacio. Curada por Ian Feldman, incluye obras de autores como William Gibson, Pat Cadigan y John Brunner, con breves descripciones y recomendaciones. Esta clasificado como "comp.graphics"

Se analizan documentos similares

In [65]:
analyze_document_similarity(most_similar_idx,
                            newsgroups_train.data,
                            newsgroups_train.target,
                            newsgroups_train.target_names,
                            cos_sim, main_idx)


Top 5 Similar Documents:
--------------------------------------------------

Analyzing Document 5745
Content:
The vote to create the proposed group, Sci.life-extension, was
affirmative.

Yes votes:    237.
No votes:      28.

What follows is a list of the people who voted, by vote ("no" or "yes").

Here are t

Document Class: sci.med
Reference Document Class: comp.graphics

Analyzing Document 9681
Content:
This is the third and final call for votes for the creation of the
newsgroup misc.health.diabetes.  A mass acknowledgement of valid votes
received as of April 19th 14:00 GMT appears at the end of this

Document Class: sci.med
Reference Document Class: comp.graphics

Analyzing Document 1128
Content:
* Attention voters:
*
* I had a problem with my mailbox on the first day of voting.
* Please check the Vote Acknowlegement (ACK) at the end of this CFV.
* If your name/address is not there, please sen

Document Class: comp.sys.ibm.pc.hardware
Reference Document Class: comp.graphics

Analy

Los documentos de votación (5745, 9681, 1128) comparten similitudes en el formato con el texto original, ya que ambos utilizan listas organizadas (votantes y libros). Sin embargo, presentan diferencias en cuanto al contenido, ya que estos textos están enfocados en la organización de grupos de noticias y procesos de votación, mientras que el documento original se centra en temas literarios, específicamente relacionados con la ciencia ficción y tecnología en libros.

Por otro lado, los documentos relacionados con gráficos y recursos (3864, 1459) tienen una mayor similitud temática, ya que pertenecen a la categoría comp.graphics, cercana al contexto técnico del documento original. No obstante, su contenido se inclina más hacia aspectos técnicos, como listados de herramientas y preguntas frecuentes, mientras que el texto original aborda la literatura y las ideas tecnológicas desde una perspectiva más conceptual y narrativa.

### 2. Entrenamiento de modelos

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

Se definen los conjuntos de entrenamiento y test. También se definen las variables objetivo y las características

In [66]:
X_train = newsgroups_train.data
y_train = newsgroups_train.target
X_test = newsgroups_test.data
y_test = newsgroups_test.target

Se define un pipeline de procesamiento con TfidfVectorizer y un modelo de Naive Bayes como clasificador.

In [67]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=1000)),
    ('model', MultinomialNB())
])

Se define el grid de hiperparámetros para ambos modelos MultinomiaNB y ComplementNB y el TfidfVectorizer

In [68]:
param_dist = [
    {
        'tfidf__max_df': np.linspace(0.5, 0.95, 5), 
        'tfidf__min_df': [1, 2, 5], 
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)], 
        'tfidf__max_features': [5000, 10000], 
        'tfidf__sublinear_tf': [True, False], 
        'model': [MultinomialNB()],
        'model__alpha': np.linspace(0.01, 1.0, 20), 
        'model__fit_prior': [True, False]
    },
    {
        'tfidf__max_df': np.linspace(0.5, 0.95, 5),
        'tfidf__min_df': [1, 2, 5],
        'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 3)], 
        'tfidf__max_features': [5000, 10000],
        'tfidf__sublinear_tf': [True, False],
        'model': [ComplementNB()],
        'model__alpha': np.linspace(0.01, 1.0, 20),
        'model__fit_prior': [True, False],
        'model__norm': [True, False]
    }
]

Se define una búsqueda aleatoria con F1-score macro como métrica de evaluación para buscar maximiazarlo.

In [69]:
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_dist, 
    n_iter=30,
    cv=5, 
    scoring='f1_macro',
    random_state=42, 
    n_jobs=2  
)

Se realiza la búsqueda de hiperparámetros y el entrenamiento del mejor modelo.

In [70]:
random_search.fit(X_train, y_train)

# Utilize the best model found by the RandomizedSearchCV
best_model = random_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro') 

print(f"Mejores Hiperparámetros: {random_search.best_params_}")
print(f"F1 Macro Score: {f1}")

Mejores Hiperparámetros: {'tfidf__sublinear_tf': True, 'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 5, 'tfidf__max_features': 10000, 'tfidf__max_df': 0.725, 'model__fit_prior': False, 'model__alpha': 0.06210526315789474, 'model': MultinomialNB()}
F1 Macro Score: 0.6654099067936965


Se ha realizado una búsqueda de hiperparámetros para los modelos MultinomialNB y ComplementNB. Se ha encontrado que la mejor configuración es la siguiente para el modelo MultinomialNB:

* tfidf__sublinear_tf: True
* tfidf__ngram_range: (1, 3)
* tfidf__min_df: 5
* tfidf__max_features: 10000
* tfidf__max_df: 0.725
* model__fit_prior: False
* model__alpha: 0.0621

Con esta configuración, se ha logrado un F1 Macro Score de 0.6654.








### 3. Vectorización de palabras

Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

Se genera la matriz témino-documento utilizando el vectorizador TF-IDF con los parámetros obtenidos en el punto anterior. Salvo el ngram_range que se ha cambiado a (1, 1) para solo considerar palabras individuales.

In [71]:
vectorizer = TfidfVectorizer(
    stop_words='english', 
    max_features=10000, 
    sublinear_tf=True, 
    ngram_range=(1, 1), 
    min_df=5, 
    max_df=0.725
)


# Fit the vectorizer on the training data
X_train_tfidf = vectorizer.fit_transform(newsgroups_train.data)

Se transpone la matriz para obtener la matriz término-documento

In [72]:
X_terms_docs = X_train_tfidf.T 
terms = vectorizer.get_feature_names_out() 

print(f"Shape of the TF-IDF matrix: {X_train_tfidf.shape}")
print(f"Number of terms: {len(terms)}")

Shape of the TF-IDF matrix: (11314, 10000)
Number of terms: 10000


Se seleccionarán cinco palabras que se consideran relevantes para el análisis de los documentos